### Notes

The models you can run here will be limited by what's implemented by the litellm library, because that's what managing the API calls.

You can see the models here: https://docs.litellm.ai/docs/



In [1]:
from functions import *

C:\Users\abiga\OneDrive\Documents\PythonScripts\llm_eval\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Parameters

In [2]:
models_dict = {
    #'claude-2.1':  "ANTHROPIC", 
    'gpt-3.5-turbo-0301': "OPENAI"
               } 
csv_file_path = '../data/prompt_target_answer_pairs.csv'
similarity_model_name = 'sentence-transformers/paraphrase-mpnet-base-v2'
temperature = "variable"
is_file_path = True
max_runs= 2
llm_evaluation_model = ['gpt-3.5', "OPENAI"]
instructions = "Please answer thoroughly: "
perturbation_model = ['gpt-4', "OPENAI"] # I recommend using a good model for perturbations otherwise it may generate the wrong number
stability_threshold= 3

pipeline = LLMAnalysisPipeline(
    input_data=csv_file_path, 
    models_dict=models_dict, 
    perturbation_model=perturbation_model, 
    llm_evaluation_model=llm_evaluation_model,
    temperature = temperature,
    max_runs= max_runs,
    is_file_path = is_file_path,
    similarity_model_name = similarity_model_name,
    instructions = instructions,
    stability_threshold = stability_threshold

)

# Run the pipeline
df_responses = pipeline.run_pipeline()



2023-12-17 09:52:50,808 - INFO - Reading prompts from CSV file.
2023-12-17 09:53:01,453 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-17 09:53:01,466 - INFO - API call successful. Model: gpt-4, Provider: OPENAI, Response: ModelResponse(id='chatcmpl-8WmnaQpHs4JR8uT1AXtVapfHLuTtH', choices=[Choices(finish_reason='stop', index=0, message=Message(content="- Can you explain the function of the cell's powerhouse and its working mechanism?\n- What is the main energy source of the cell and how does it operate?\n- Can you describe the cell's energy factory and how it functions?\n- What is the cell's main energy producer and how does it work?\n- What is the energy center of the cell and how does it perform its functions?\n- Can you tell me about the cell's energy hub and how it works?\n- What is the energy generator of the cell and how does it function?\n- What is the primary energy supplier of the cell and how does it operate?\n- Can you explai

0


2023-12-17 09:54:17,254 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-17 09:54:17,259 - INFO - API call successful. Model: gpt-3.5-turbo-0301, Provider: OPENAI, Response: ModelResponse(id='chatcmpl-8Wmosnun2rXPHcvV4xNJM8nWPq59n', choices=[Choices(finish_reason='stop', index=0, message=Message(content='The powerhouse of the cell is the mitochondria. It is responsible for producing energy in the form of ATP (adenosine triphosphate) through a process called cellular respiration. \n\nMitochondria have an outer membrane and an inner membrane, with a space in between called the intermembrane space. The inner membrane is highly folded, forming structures called cristae. These folds increase the surface area of the inner membrane, allowing for more ATP production.\n\nThe process of cellular respiration occurs in three stages: glycolysis, the citric acid cycle, and oxidative phosphorylation. \n\nDuring glycolysis, glucose is broken down into tw

1


2023-12-17 09:54:25,316 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-17 09:54:25,323 - INFO - API call successful. Model: gpt-3.5-turbo-0301, Provider: OPENAI, Response: ModelResponse(id='chatcmpl-8Wmp1LoPBl3dJ8XfO0HHyyKliba7g', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes, I can explain the role and function of the cell's powerhouse.\n\nThe cell's powerhouse is the mitochondria. The mitochondria are responsible for producing ATP (adenosine triphosphate), which is the primary source of energy for the cell. ATP is produced through a process called cellular respiration, which occurs within the mitochondria.\n\nThe mitochondria are double-membrane organelles that contain their own DNA and ribosomes. They are involved in several important cellular processes, including the regulation of calcium levels within the cell, the synthesis of certain lipids and amino acids, and the initiation of programmed cell deat

...

2023-12-17 09:55:36,638 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-17 09:55:36,642 - INFO - API call successful. Model: gpt-3.5-turbo-0301, Provider: OPENAI, Response: ModelResponse(id='chatcmpl-8WmqDse89IaYFzWIp2jaXd5dNJft4', choices=[Choices(finish_reason='stop', index=0, message=Message(content='As an AI language model, I cannot find any credible source or study that mentions the specific particle or molecule that keeps the membrane of hover frogs stable. It is possible that this information is not currently known or studied extensively. However, the membranes of hover frogs (also known as flying frogs) are unique adaptations that allow them to glide through the air over short distances. These membranes act as wings and are supported by cartilage and connective tissue. The process of flight in hover frogs involves a combination of gliding, falling, and hopping. The exact mechanisms that enable their flight remain the subject of s

0


2023-12-17 09:55:43,646 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-17 09:55:43,649 - INFO - API call successful. Model: gpt-3.5-turbo-0301, Provider: OPENAI, Response: ModelResponse(id='chatcmpl-8WmqH71AjzPMKYTMslSS9PoSGNhuZ', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Astroflora, also known as space plants, are plants that are grown in space or in low light environments. These plants have adapted to low light conditions by developing unique mechanisms for photosynthesis.\n\nOne way that astroflora perform photosynthesis in low light is by increasing the efficiency of their light-harvesting systems. They have developed specialized pigments that can absorb light at longer wavelengths, such as red and blue light, which are more abundant in low light conditions. These pigments, such as phytochrome and cryptochrome, allow the plants to capture more light energy and convert it into chemical energy through ph

1


2023-12-17 09:55:52,134 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-17 09:55:52,138 - INFO - API call successful. Model: gpt-3.5-turbo-0301, Provider: OPENAI, Response: ModelResponse(id='chatcmpl-8WmqQWTOanjLxvhavnbGbzv6GgEbE', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Astroflora, also known as space plants or astrobotany, are plants that are grown in space or low light environments. These plants require light for photosynthesis, which is the process by which they convert light energy into chemical energy that they can use for growth and survival.\n\nIn low light conditions, astroflora have adapted several strategies to perform photosynthesis. Here are a few examples:\n\n1. Enhanced Pigment Production: Astroflora can produce more pigment to capture light energy in low light environments. One pigment that helps plants absorb light is chlorophyll, which is responsible for the green color of plants. Some a

In [13]:
df_responses

,model,original_prompt,response,temperature,actual_prompt,run_number,similarity_score,keyword_score,llm_rating,true_or_false,keywords
0,gpt-3.5-turbo-0301,What is the powerhouse of the cell and how doe...,The powerhouse of the cell is the mitochondria...,0.066700,What is the powerhouse of the cell and how doe...,0,0.872075,0.642857,0.9,True,"[mitochondrion, mitochondria, atp, adenosine t..."
1,gpt-3.5-turbo-0301,What is the powerhouse of the cell and how doe...,"Yes, I can explain the role and function of th...",0.902689,- Can you explain the role and function of the...,1,0.835412,0.571429,0.9,True,"[mitochondrion, mitochondria, atp, adenosine t..."
2,gpt-3.5-turbo-0301,What is the basic unit of life and what does t...,The key unit of life is the cell. Cells are th...,0.837901,- What is the key unit of life and what is its...,0,0.778232,0.133333,0.8,True,"[cell, biology, prokaryotic cells, eukaryotic ..."
3,gpt-3.5-turbo-0301,What is the basic unit of life and what does t...,The fundamental unit of life is the cell. A ce...,0.903338,- What is the fundamental unit of life and can...,1,0.833760,0.400000,0.7,True,"[cell, biology, prokaryotic cells, eukaryotic ..."
4,gpt-3.5-turbo-0301,What do genes consist of and how do they work?,Genes are segments of DNA (deoxyribonucleic ac...,0.575674,- What makes up genes and how do they function?,0,0.863344,0.615385,1.0,True,"[genes, dna, deoxyribonucleic acid, nucleotide..."
5,gpt-3.5-turbo-0301,What do genes consist of and how do they work?,Genes are made up of DNA (deoxyribonucleic aci...,0.090602,- What are genes made of and how do they operate?,1,0.820403,0.692308,0.9,True,"[genes, dna, deoxyribonucleic acid, nucleotide..."
6,gpt-3.5-turbo-0301,Which organelle contains the cell's futurogeni...,The name of the organelle that holds the cell'...,0.898714,- What is the name of the organelle that holds...,0,0.478814,0.000000,0.0,False,"[chronocytum, futurogenic material, temporal a..."
7,gpt-3.5-turbo-0301,Which organelle contains the cell's futurogeni...,"Yes, the organelle that carries the material t...",0.858036,- Can you identify the organelle that carries ...,1,0.454238,0.000000,0.2,False,"[chronocytum, futurogenic material, temporal a..."
8,gpt-3.5-turbo-0301,What particle stabilizes the membrane of hover...,"As an AI language model, I do not have access ...",0.705843,- Can you identify the particle that ensures t...,0,0.587539,0.111111,0.1,False,"[levitonium, hover frogs, membrane stabilizati..."
9,gpt-3.5-turbo-0301,What particle stabilizes the membrane of hover...,"As an AI language model, I cannot find any cre...",0.973983,- What is the specific particle that keeps the...,1,0.709304,0.111111,0.2,False,"[levitonium, hover frogs, membrane stabilizati..."


In [14]:
aggregate_best_scores(df_responses, "similarity_score")

,model,original_prompt,actual_prompt,response,true_or_false,similarity_score
11,gpt-3.5-turbo-0301,How do astroflora perform photosynthesis in lo...,How do astroflora perform photosynthesis in lo...,"Astroflora, also known as space plants or astr...",False,0.847789
4,gpt-3.5-turbo-0301,What do genes consist of and how do they work?,- What makes up genes and how do they function?,Genes are segments of DNA (deoxyribonucleic ac...,True,0.863344
3,gpt-3.5-turbo-0301,What is the basic unit of life and what does t...,- What is the fundamental unit of life and can...,The fundamental unit of life is the cell. A ce...,True,0.833760
0,gpt-3.5-turbo-0301,What is the powerhouse of the cell and how doe...,What is the powerhouse of the cell and how doe...,The powerhouse of the cell is the mitochondria...,True,0.872075
9,gpt-3.5-turbo-0301,What particle stabilizes the membrane of hover...,- What is the specific particle that keeps the...,"As an AI language model, I cannot find any cre...",False,0.709304
6,gpt-3.5-turbo-0301,Which organelle contains the cell's futurogeni...,- What is the name of the organelle that holds...,The name of the organelle that holds the cell'...,False,0.478814


In [5]:
aggregate_best_scores(df_responses, "similarity_score")

,model,original_prompt,actual_prompt,response,true_or_false,similarity_score
11,gpt-3.5-turbo-0301,How do astroflora perform photosynthesis in lo...,How do astroflora perform photosynthesis in lo...,"Astroflora, also known as space plants or astr...",False,0.847789
4,gpt-3.5-turbo-0301,What do genes consist of and how do they work?,- What makes up genes and how do they function?,Genes are segments of DNA (deoxyribonucleic ac...,True,0.863344
3,gpt-3.5-turbo-0301,What is the basic unit of life and what does t...,- What is the fundamental unit of life and can...,The fundamental unit of life is the cell. A ce...,True,0.833760
0,gpt-3.5-turbo-0301,What is the powerhouse of the cell and how doe...,What is the powerhouse of the cell and how doe...,The powerhouse of the cell is the mitochondria...,True,0.872075
9,gpt-3.5-turbo-0301,What particle stabilizes the membrane of hover...,- What is the specific particle that keeps the...,"As an AI language model, I cannot find any cre...",False,0.709304
6,gpt-3.5-turbo-0301,Which organelle contains the cell's futurogeni...,- What is the name of the organelle that holds...,The name of the organelle that holds the cell'...,False,0.478814


In [6]:
aggregate_best_scores(df_responses, 'keyword_score')

,model,original_prompt,actual_prompt,response,true_or_false,keyword_score
10,gpt-3.5-turbo-0301,How do astroflora perform photosynthesis in lo...,How do astroflora perform photosynthesis in lo...,"Astroflora, also known as space plants, are pl...",False,0.363636
5,gpt-3.5-turbo-0301,What do genes consist of and how do they work?,- What are genes made of and how do they operate?,Genes are made up of DNA (deoxyribonucleic aci...,True,0.692308
3,gpt-3.5-turbo-0301,What is the basic unit of life and what does t...,- What is the fundamental unit of life and can...,The fundamental unit of life is the cell. A ce...,True,0.400000
0,gpt-3.5-turbo-0301,What is the powerhouse of the cell and how doe...,What is the powerhouse of the cell and how doe...,The powerhouse of the cell is the mitochondria...,True,0.642857
8,gpt-3.5-turbo-0301,What particle stabilizes the membrane of hover...,- Can you identify the particle that ensures t...,"As an AI language model, I do not have access ...",False,0.111111
6,gpt-3.5-turbo-0301,Which organelle contains the cell's futurogeni...,- What is the name of the organelle that holds...,The name of the organelle that holds the cell'...,False,0.000000


In [7]:
aggregate_best_scores(df_responses, 'llm_rating')

,model,original_prompt,actual_prompt,response,true_or_false,llm_rating
10,gpt-3.5-turbo-0301,How do astroflora perform photosynthesis in lo...,How do astroflora perform photosynthesis in lo...,"Astroflora, also known as space plants, are pl...",False,0.8
4,gpt-3.5-turbo-0301,What do genes consist of and how do they work?,- What makes up genes and how do they function?,Genes are segments of DNA (deoxyribonucleic ac...,True,1.0
2,gpt-3.5-turbo-0301,What is the basic unit of life and what does t...,- What is the key unit of life and what is its...,The key unit of life is the cell. Cells are th...,True,0.8
0,gpt-3.5-turbo-0301,What is the powerhouse of the cell and how doe...,What is the powerhouse of the cell and how doe...,The powerhouse of the cell is the mitochondria...,True,0.9
9,gpt-3.5-turbo-0301,What particle stabilizes the membrane of hover...,- What is the specific particle that keeps the...,"As an AI language model, I cannot find any cre...",False,0.2
7,gpt-3.5-turbo-0301,Which organelle contains the cell's futurogeni...,- Can you identify the organelle that carries ...,"Yes, the organelle that carries the material t...",False,0.2


In [8]:
df_responses.columns

Index(['model', 'original_prompt', 'response', 'temperature', 'actual_prompt',
       'run_number', 'similarity_score', 'keyword_score', 'llm_rating',
       'true_or_false', 'keywords'],
      dtype='object')

In [9]:
def aggregate_all_best(df, methods):
    print("Starting aggregation...")  # Debugging print
    aggregated_dfs = []

    for method in methods:
        print(f"Aggregating method: {method}")  # Debugging print
        if method not in df.columns:
            print(f"Warning: Column '{method}' not found in DataFrame")  # Debugging print
            continue

        df_method_best = df.loc[df.groupby(['model', 'original_prompt'])[method].idxmax()]
        df_method_best = df_method_best[['model', 'original_prompt', 'true_or_false', method]]
        df_method_best.rename(columns={method: 'score'}, inplace=True)
        df_method_best['method'] = method
        aggregated_dfs.append(df_method_best)

    df_best_scores = pd.concat(aggregated_dfs, ignore_index=True)
    return df_best_scores

# Then call the function
methods = ['llm_rating', 'keyword_score', 'similarity_score']
df_best_scores = aggregate_all_best(df_responses, methods)
df_best_scores

Starting aggregation...
Aggregating method: llm_rating
Aggregating method: keyword_score
Aggregating method: similarity_score


,model,original_prompt,true_or_false,score,method
0,gpt-3.5-turbo-0301,How do astroflora perform photosynthesis in lo...,False,0.800000,llm_rating
1,gpt-3.5-turbo-0301,What do genes consist of and how do they work?,True,1.000000,llm_rating
2,gpt-3.5-turbo-0301,What is the basic unit of life and what does t...,True,0.800000,llm_rating
3,gpt-3.5-turbo-0301,What is the powerhouse of the cell and how doe...,True,0.900000,llm_rating
4,gpt-3.5-turbo-0301,What particle stabilizes the membrane of hover...,False,0.200000,llm_rating
5,gpt-3.5-turbo-0301,Which organelle contains the cell's futurogeni...,False,0.200000,llm_rating
6,gpt-3.5-turbo-0301,How do astroflora perform photosynthesis in lo...,False,0.363636,keyword_score
7,gpt-3.5-turbo-0301,What do genes consist of and how do they work?,True,0.692308,keyword_score
8,gpt-3.5-turbo-0301,What is the basic unit of life and what does t...,True,0.400000,keyword_score
9,gpt-3.5-turbo-0301,What is the powerhouse of the cell and how doe...,True,0.642857,keyword_score


In [15]:
import plotly.express as px

# Calculate the count for each combination of method, score, true_or_false, and model
df_best_scores['count'] = df_best_scores.groupby(['method', 'score', 'true_or_false', 'model'])['model'].transform('count')

# Creating the scatter plot
fig = px.scatter(df_best_scores, x='method', y='score', color='true_or_false', 
                 facet_col='model', facet_col_wrap=2, 
                 hover_data=['original_prompt'], category_orders={'true_or_false': ['True', 'False']},
                 size='count', size_max=15)

# Customizing the layout
fig.update_layout(title='Comparison of Evaluation Methods for True vs False Prompts',
                  xaxis_title='Method', yaxis_title='Score',
                  legend_title_text='True or False')

In [16]:
def calculate_spread(df_responses):
    grouped = df_responses.groupby(['model', 'original_prompt'])
    spread_df = grouped.agg({'similarity_score': 'std', 'keyword_score': 'std', 'llm_rating': 'std'}).reset_index()
    return spread_df

calculate_spread(df_responses)

,model,original_prompt,similarity_score,keyword_score,llm_rating
0,gpt-3.5-turbo-0301,How do astroflora perform photosynthesis in lo...,0.003274,0.000000,0.000000
1,gpt-3.5-turbo-0301,What do genes consist of and how do they work?,0.030364,0.054393,0.070711
2,gpt-3.5-turbo-0301,What is the basic unit of life and what does t...,0.039264,0.188562,0.070711
3,gpt-3.5-turbo-0301,What is the powerhouse of the cell and how doe...,0.025924,0.050508,0.000000
4,gpt-3.5-turbo-0301,What particle stabilizes the membrane of hover...,0.086101,0.000000,0.070711
5,gpt-3.5-turbo-0301,Which organelle contains the cell's futurogeni...,0.017378,0.000000,0.141421


In [12]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

# Assuming df_responses contains columns 'original_prompt', 'similarity_scores', 'keyword_scores', 'llm_ratings'
prompts = df_responses['original_prompt'].unique()

# Create subplot titles
subplot_titles = [("", prompt, "") for prompt in prompts]

# Flatten the list of subplot titles
subplot_titles_flat = [title for trio in subplot_titles for title in trio]

# Create subplots with the specified titles
fig = make_subplots(rows=len(prompts), cols=3, subplot_titles=subplot_titles_flat, horizontal_spacing=0.02, vertical_spacing=0.05)

# Define colors for each method
colors = {'Similarity': 'blue', 'Keyword': 'green', 'LLM Rating': 'red'}

# Function to slightly adjust x positions for swarm plot effect
def jitter_x_positions(length, center_position, width=0.2):
    return np.random.uniform(center_position - width/2, center_position + width/2, length)

# Add traces for each method with specified colors
for i, prompt in enumerate(prompts, start=1):
    # Filter data for each prompt
    filtered_data = df_responses[df_responses['original_prompt'] == prompt]
    
    # Similarity Scores
    fig.add_trace(go.Scatter(
        x=jitter_x_positions(len(filtered_data), 1), 
        y=filtered_data['similarity_score'], 
        mode='markers', 
        marker=dict(size=10, opacity=0.5, color=colors['Similarity']),
        name="Similarity"
    ), row=i, col=1)

    # Keyword Scores
    fig.add_trace(go.Scatter(
        x=jitter_x_positions(len(filtered_data), 2), 
        y=filtered_data['keyword_score'], 
        mode='markers', 
        marker=dict(size=10, opacity=0.5, color=colors['Keyword']),
        name="Keyword"
    ), row=i, col=2)

    # LLM Ratings
    fig.add_trace(go.Scatter(
        x=jitter_x_positions(len(filtered_data), 3), 
        y=filtered_data['llm_rating'], 
        mode='markers', 
        marker=dict(size=10, opacity=0.5, color=colors['LLM Rating']),
        name="LLM Rating"
    ), row=i, col=3)

# Update layout
fig.update_layout(height=200*len(prompts), width=1200, title_text="Evaluation Scores by Prompt and Method", showlegend=False)
fig.update_yaxes(range=[-.1, 1.1])  # Set y-axis range for all subplots
fig.update_xaxes(showticklabels=False)

fig.show()
